</div>
<div align="left">
  <img src="img/abstract.png" width="400" alt="Funny little diagram">
  <p><em> Evolve nodes, evolve plans, and learn from the best performing ones.</em></p>
</div>
<div align="center">
</em></p>
</div>

#### Node Initialization (Refactoring ...)

In [1]:
from methods.llm import get_async_vllm_endpoint
import os 

# Unlimited LLM endpoints
endpoint_id = "vllm-4qqnnwfdggvjba"
api_key = "rpa_EPOJED42G59S80Y6SKMCOI330EQU4JPPMKV2UD2W7j0uku"
get_endpoint_response = get_async_vllm_endpoint(endpoint_id, api_key)

Could not load vllm class, check CUDA support and GPU RAM size


In [2]:
from methods.meta_prompt import MetaPrompt, PromptMode
from methods.evolnode import EvolNode
from methods.llm import get_groq_response, get_claude_response

# Code + Compilor Task
# mp = MetaPrompt("Search for age of a celebrity.", "get_celeb_age", ["name"], ["age"], ["str"], ["int"], PromptMode.CODE)
# Prompt + LLM Task
mp = MetaPrompt("Get the age of a celebrity.", "get_celeb_age", ["name"], ["age"], ["str"], ["int"], PromptMode.PROMPT) # 

test_cases = [
    ({"name": "Dilireba"}, {"age": 32}),
    ({"name": "ChengXiao"}, {"age": 26})
]

test_inputs = [c[0] for c in test_cases]

node = EvolNode(mp, None, None, get_response=get_endpoint_response, test_cases=test_cases) # setting manual test cases

node.evolve("i1", replace=True, batch_size=40, num_runs=2, print_summary=True) # Scale up batch size


input_dict = {"name": "Dilireba"}
output_dict = node(input_dict, max_attempts=6) # Batch Inference with vLLM

# node.get_response = get_groq_response # fast sequential inference
# output_dict = node(input_dict, max_attempts=6, batch_inference=False)
 
print("Output dict: ", output_dict)

Processing LLM queries: 100%|██████████| 40/40 [00:22<00:00,  1.77it/s]


 :: Total time elapsed: 22.57s, 0 errors


Processing LLM queries: 100%|██████████| 39/39 [00:33<00:00,  1.15it/s]


 :: Total time elapsed: 33.82s, 0 errors


Processing LLM queries: 0it [00:00, ?it/s]


 :: Total time elapsed: 0.00s, 0 errors
🏆 Best Code Performance Summary 🏆
  ⚡ Structural fitness: 0.33
  🎯 Functional fitness: 1.00
  ⭐ Global fitness:     0.67
  🔄 Compiled solutions:        31
  ⏱️ Time breakdown:
     :: Query time: 33.71s
     :: Evolution time: 22.63s
     :: Evaluation time: 91.16s
     :: Total time: 147.50s


📊 Code 0: Fitness: 33.3%
--------------------------------------------------------------------------------
❌ Error Messages:
Failed to parse LLM response -- JsonDecodeError : 
Expecting value: line 1 column 1 (char 0)AstLiteralError : 
invalid syntax (<unknown>, line 1)


📊 Code 1: Fitness: 33.3%
--------------------------------------------------------------------------------
❌ Error Messages:
Failed to parse LLM response -- JsonDecodeError : 
Expecting value: line 1 column 1 (char 0)AstLiteralError : 
invalid syntax (<unknown>, line 1)


📊 Code 2: Fitness: 0.0%
--------------------------------------------------------------------------------
❌ Error Message

Processing LLM queries: 100%|██████████| 6/6 [00:06<00:00,  1.09s/it]

 :: Total time elapsed: 6.52s, 0 errors
Output dict:  {'age': 32}


</div>
<div align="center">
  <img src="img/Project-Nirvana-evolve.gif" width="500" alt="Fourier reconstruction convergence">
  <p><em> Evolve a population of nodes. </em></p>
</div>

In [4]:
# Population building phase ... 
from methods.llm import get_groq_response, get_claude_response
from methods.meta_prompt import MetaPrompt, PromptMode
from methods.population import Evolution

mp = MetaPrompt("Get the age of a celebrity.", "get_celeb_age", ["name"], ["age"], ["str"], ["int"], PromptMode.PROMPT) # 

test_cases = [
    ({"name": "Dilireba"}, {"age": 32}),
    ({"name": "ChengXiao"}, {"age": 26})
]

evo = Evolution(pop_size=20, meta_prompt=mp, get_response=get_endpoint_response, 
                test_cases=test_cases, max_attempts=3, num_eval_runs=2,
                load=True)

strategies = ["m2"] # ["i1", "i1", "m2", "e2"]
evo.get_offspring(strategies)

evo.chat("How effective is the current evolution strategy? What improvement has it made in terms of fitness, and in terms of the implementation?",
         get_claude_response) 

# code-based check 
print(evo.population_info)

Processing LLM queries: 100%|██████████| 20/20 [00:22<00:00,  1.11s/it]


 :: Total time elapsed: 22.27s, 0 errors


Processing LLM queries: 100%|██████████| 22/22 [00:37<00:00,  1.71s/it]


 :: Total time elapsed: 37.61s, 0 errors


Processing LLM queries: 0it [00:00, ?it/s]


 :: Total time elapsed: 0.00s, 0 errors
🏆 Best Code Performance Summary 🏆
  ⚡ Structural fitness: 0.33
  🎯 Functional fitness: 1.00
  ⭐ Global fitness:     0.67
  🔄 Compiled solutions:        17
  ⏱️ Time breakdown:
     :: Query time: 2.59s
     :: Evolution time: 22.32s
     :: Evaluation time: 68.43s
     :: Total time: 93.33s


📊 Code 0: Fitness: 8.3%
--------------------------------------------------------------------------------
❌ Error Messages:
Input: {'name': 'ChengXiao'}, prediction is not aligned with expected output, Expected: {'age': 26} Predicted: {'birth_date': {'answerBox': {'snippet': 'Cheng Xiao (Chinese: 程潇; Korean: 성소, born July 15, 1998) is a Chinese singer, dancer, and actress.', 'snippetHighlighted': ['July 15, 1998'], 'title': 'Cheng Xiao - Wikipedia', 'link': 'https://en.wikipedia.org/wiki/Cheng_Xiao'}, 'Search Result': [{'title': 'Cheng Xiao (WJSN) profile, age & facts (2024 updated) | kpopping', 'link': 'https://kpopping.com/profiles/idol/Cheng-Xiao', 'snippe


</div>
<div align="center">
  <img src="https://github.com/user-attachments/assets/af98faeb-66d6-4278-af86-67d668d1954e" width="900" alt="Fourier reconstruction convergence">
  <p><em> Plan, and evolve the plans. </em></p>
</div>


In [3]:
from methods.llm import get_claude_response, get_groq_response
from methods.diagram import visualize_plan_dict
from methods.meta_prompt import MetaPlan
from methods.evolnode import PlanNode


# Initialize PlanNode 
mp = MetaPlan("Get the age of celebrity.", "get_celeb_age", ["name"], ["age"], ["str"], ["int"])
plan = PlanNode(mp, get_endpoint_response)

# i1 evolution of plan
plan_dicts, err_msg = plan.evolve_plan_dict(method="i1", batch_size=10) # Batch_size of 100 gives no slow-down

visualize_plan_dict(plan.plan_dict, plan.meta_prompt.task) # most simpliest plan

# Manual input on main-node test cases 
main_test_cases = [
    ({"name": "Dilireba"}, {"age": 32}),
    ({"name": "ChengXiao"}, {"age": 26})
]

is_success, err_msg = plan.spawn_test_cases(main_test_cases) #  pinned test cases generation
# plan.spawn_test_cases_majority(main_test_cases) # multi-agent test cases generation (need some benchmarking to compare quality)

 :: Evolving 10 plans in parallel...


Processing LLM queries:   0%|          | 0/10 [00:00<?, ?it/s]

In [6]:
# plan.evolve_sub_nodes() # Completely stuck in the first call, debugging ... 

from methods.evolnode import EvolNode
from methods.meta_prompt import MetaPrompt, PromptMode

self = plan 

# 1. Did we skip existing nodes? Yes, those node has code & fitness and is skipped.
# 2. Did we make the name compatible with slightly off input?

for i, node_dict in enumerate(self.plan_dict["nodes"]):
    meta_prompt = MetaPrompt(
        task=node_dict["task"],
        func_name=node_dict["name"],
        inputs=node_dict["inputs"],
        outputs=node_dict["outputs"],
        input_types=node_dict["input_types"],
        output_types=node_dict["output_types"],
        mode=PromptMode((node_dict.get("mode", "code")).lower())
    )
    test_cases = self.test_cases_dict[node_dict["name"]]
    if "fitness" in node_dict and "code" in node_dict: 
        node = EvolNode(meta_prompt, node_dict["code"], node_dict["reasoning"], get_response=self.get_response, test_cases=test_cases, fitness=node_dict["fitness"])
    else:
        node = EvolNode(meta_prompt, None, None, get_response=self.get_response, test_cases=test_cases)
        print(f"🎲 :: Evolving {node.meta_prompt.func_name} ... ({i+1}/{len(self.plan_dict['nodes'])})")
        node.evolve("i1", replace=True, max_tries=2, num_runs=2, batch_size=20) # It's funny how 30+ sec could elapse before llm inference ... (collecting prompts ?? wtf is taking so long ??)
    self.nodes.append(node)

🎲 :: Evolving parse_birthdate ... (2/3)


Processing LLM queries: 100%|██████████| 20/20 [00:19<00:00,  1.01it/s]


 :: Total time elapsed: 19.83s, 0 errors


Processing LLM queries: 100%|██████████| 14/14 [00:07<00:00,  1.78it/s]


 :: Total time elapsed: 7.89s, 0 errors


TypeError: 'float' object is not iterable

In [9]:
# node.evolve(method, replace=replace, max_tries=max_tries, num_runs=num_runs, batch_size=batch_size) # missing required input parameters: 'result' --> break down 

replace = True
method = "i1"
parents = []
batch_size = 20
num_runs = 2
max_tries = 2
self = node
feedback = ""

self.query_nodes(ignore_self=replace, self_func_name=self.meta_prompt.func_name) # look for relevant nodes in the library, go down

reasonings, codes = self._evolve(method, parents, batch_size=batch_size) # Issue #2. duplicate relevant functions after compilation, go down 

# responses = self._get_evolve_response(method, parents, feedback, batch_size)

# from methods.evolnode import parse_evol_response, compile_code_with_references

# response = responses[0]
# reasoning, code = parse_evol_response(response)
# # code = compile_code_with_references(code, self.referrable_function_dict) # deal with node references | go down


Processing LLM queries: 100%|██████████| 20/20 [00:18<00:00,  1.10it/s]


 :: Total time elapsed: 18.11s, 0 errors


In [19]:
self = node 
self.reasonings = reasonings
self.codes = codes

# Layer 1
# fitness_per_code, errors_per_code, global_summary = self._evaluate_fitness(codes=codes, max_tries=max_tries, num_runs=num_runs) # error here 'float' object is being looped over ?


# Layer 2
from methods.evolnode import check_alignment_parallel, combine_errors

if len(codes) == 0: 
    codes = [self.code] 

if test_cases is None:
    test_cases = self.test_cases

if self.meta_prompt.mode == PromptMode.PROMPT:
    num_runs = min(2, num_runs) # sanity check against stochastic nature of prompt-based node
        
test_inputs = [case[0] for case in test_cases]

if self.meta_prompt.mode == PromptMode.CODE: 
    output_per_code_per_test, errors_per_code_per_test = self.call_code_function_parallel(test_inputs, codes)
elif self.meta_prompt.mode == PromptMode.PROMPT:
    output_per_code_per_test, errors_per_code_per_test = self.call_prompt_function_parallel(test_inputs, codes, max_tries)
    
# # alignment checking
# test_inputs = [case[0] for case in test_cases]
# target_outputs = [case[1] for case in test_cases]
# score_per_code_per_test, evaluate_errors_per_code_per_test = check_alignment_parallel(output_per_code_per_test, test_inputs, target_outputs, 
#                                                                                         self.get_response, batch_size=num_runs) # here is the issue 


# Layer 3 

# make sure to combined score obtained from both steps (average of both)
from methods.evolnode import _check_alignment_with_metric_parallel, _check_alignment_with_llm_parallel, combine_scores, defaultdict

errors_per_code_per_test = defaultdict(lambda: defaultdict(list))

# Get scores from metric-based alignment check
metric_scores, errors_per_code_per_test = _check_alignment_with_metric_parallel(output_per_code_per_test, errors_per_code_per_test, test_inputs, target_outputs)

# Get scores from LLM-based alignment check 
llm_scores, errors_per_code_per_test = _check_alignment_with_llm_parallel(output_per_code_per_test, errors_per_code_per_test, test_inputs, target_outputs,
                                                                            self.get_response, batch_size)

score_per_code_per_test = combine_scores(llm_scores, metric_scores)

Processing LLM queries:   1%|          | 1/160 [00:01<03:28,  1.31s/it]

Error in completion: 'NoneType' object has no attribute 'choices'


Processing LLM queries: 100%|██████████| 160/160 [00:07<00:00, 20.07it/s]

 :: Total time elapsed: 7.97s, 1 errors


TypeError: 'float' object is not iterable